In [5]:
import numpy as np
import pandas as pd

df = pd.read_csv('s2-in.txt', sep='\t')
df.head()

,Book,Chapter,Verse,Text
0,1Rg,1,1,"WMLK> DWJD S>B W<L BC""NJ> WMKSJN HWW LH BLBWC""..."
1,1Rg,1,2,"W>MRW LH <B""DWHJ H> <B""DJK QDMJK NB<WN LMRN ML..."
2,1Rg,1,3,WB<W <LJMT> DCPJR> BKLH TXWM> D>JSRJL W>CKXW L...
3,1Rg,1,4,W<LJMT> CPJR> HWT BXZWH VB WHWT LMLK> MCMCNJT>...
4,1Rg,1,5,W>DWNJ> BR XGJT MTRWRB W>MR >N> >MLK W<BD LH M...


In [10]:
vocab = {}
i = 1

with open('s2-in.txt') as f:
    x = f.read().lower().split()

for word in x:
    if word in vocab:
        continue
    else:
        vocab[word]=i
        i+=1

print(vocab)

{'book': 1, 'chapter': 2, 'verse': 3, 'text': 4, '1rg': 5, '1': 6, 'wmlk>': 7, 'dwjd': 8, 's>b': 9, 'w<l': 10, 'bc"nj>': 11, 'wmksjn': 12, 'hww': 13, 'lh': 14, 'blbwc">': 15, 'wl>': 16, 'cxn': 17, '2': 18, 'w>mrw': 19, '<b"dwhj': 20, 'h>': 21, '<b"djk': 22, 'qdmjk': 23, 'nb<wn': 24, 'lmrn': 25, 'mlk>': 26, '<ljmt>': 27, 'btwlt>': 28, 'wtqwm': 29, 'qdm': 30, 'wthw>': 31, 'mcmcnjt>': 32, 'wtckb': 33, 'b<wbk': 34, 'wncxn': 35, '3': 36, 'wb<w': 37, 'dcpjr>': 38, 'bklh': 39, 'txwm>': 40, 'd>jsrjl': 41, 'w>ckxw': 42, 'l>bjcg': 43, 'cjlwmjt>': 44, 'w>jtjwh': 45, 'lmlk>': 46, '4': 47, 'w<ljmt>': 48, 'cpjr>': 49, 'hwt': 50, 'bxzwh': 51, 'vb': 52, 'whwt': 53, 'wmcmc>': 54, 'l>': 55, 'jd<h': 56, '5': 57, 'w>dwnj>': 58, 'br': 59, 'xgjt': 60, 'mtrwrb': 61, 'w>mr': 62, '>n>': 63, '>mlk': 64, 'w<bd': 65, 'mr"kbt>': 66, 'wpr"c>': 67, 'wxmcjn': 68, 'gbr"jn': 69, 'drhvjn': 70, 'qdmwhj': 71, '6': 72, 'k>>': 73, 'bh': 74, '>bwhj': 75, 'mn': 76, 'jwm"whj': 77, 'mvl': 78, 'mn>': 79, 'hkn>': 80, '<bd': 81, '